# Proyecto N°1: Clasificación con método _Bag of Features_

This project will be evaluated in class groups. The evaluation metrics are based on your presentation and code provided in your attachments. Your solutions to the following problem should include commented source code written in Python. You can use additional modules if necessary. Analyze your results. If you discover something interesting, let us know!

The past decade has seen the growing popularity of Bag of Features (BoF) approaches to many computer vision tasks, including image classification, video search, robot localization, and texture recognition. Part of the appeal is simplicity. BoF methods are based on orderless collections of quantized local image descriptors; they discard spatial information and are therefore conceptually and computationally simpler than many alternative methods [here](https://arxiv.org/pdf/1101.3354.pdf).

For this project, you have to implement the Bag of Features representation for a classification problem. To accomplish this, execute the following tasks in the given order:

1. Download CIFAR-10 dataset [here](https://www.kaggle.com/c/cifar-10/data). You have to select at least 4 classes to classify.
2. For each image, compute Bag of features descriptors.
3. Select a classifier using the BoF as input data, Options are Neural Network and KNN
4. Evaluate the classifier using the testing set.
5. Compute performance metrics creating a confusion matrix.

For **task #3**, you have to read and implement the approach presented [here](https://arxiv.org/pdf/1101.3354.pdf), and you can look for additional information on internet. For **task #6** you should measure model performance with unseen data for this classification problem.

### 1. Importación de módulos y librerias.

In [1]:
import cv2
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

### 2. Selección de imagenes de entrenamiento (4 categorías).

#### 2.A Lectura del archivo de etiquetas .csv (_Comma separated values_)

In [2]:
# Abrimos el archivo e imprimimos la cabecera para verificar.
filename = 'trainLabels.csv'
labels = pd.read_csv(filename)
print(labels.head())

   id       label
0   1        frog
1   2       truck
2   3       truck
3   4        deer
4   5  automobile


#### 2.B Lectura de carpeta con imagenes

In [3]:
# Ruta de la carpeta con los archivos
path = 'train'
train_set = os.listdir(path)
print('Lista de imagenes de entrenamiento - Clase "{}" | Tamaño: {}'.format(type(train_set).__name__, len(train_set)))
print(train_set[0:5])

Lista de imagenes de entrenamiento - Clase "list" | Tamaño: 50000
['1.png', '10.png', '100.png', '1000.png', '10000.png']


#### 2.C Selección de imagenes en base a etiquetas elegidas.

In [4]:
# Lista de categorias seleccionadas
categorias = ['cat', 'frog', 'truck', 'ship']

# Filtro en base a las categorias.
train_list = labels.loc[labels['label'].isin(categorias)]['id'].tolist()
print('Tamaño del set de entrenamiento en base a categorías: {}'.format(len(train_list)))

Tamaño del set de entrenamiento en base a categorías: 20000


### 3. _Bag of Features_ (Descriptor)

In [46]:
# Creamos nuestra lista de vectores con los descriptores.
des_array = []

for img_item in train_list[0:20] :
    # Creamos nuestro elemento SIFT por cada imagen
    img  = cv2.imread(os.path.join(path, str(img_item)+'.png'))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculo de los descriptores y los puntos clave (Keypoints)
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(gray,None)
    # Guardamos dicho array en la lista
    des_array.append(des)

# Matriz n x 128 por cada descriptor
des_shape = [v.shape[0] for v in des_array]
print('Tamaño aproximado de descriptores por imagen: {}'.format(np.mean(des_shape)))

Tamaño aproximado de descriptores por imagen: 14.8


In [47]:
# Organizamos todos los descriptores en una sola matriz.
des_stack = np.vstack(des_array)
print('Tamaño total de descriptores: {}'.format(des_stack.shape))

Tamaño total de descriptores: (296, 128)


### 4. Clasificador por método Kmeans

#### 4.A Función para los centroides